In [1]:
import bcolz
import numpy as np
import pickle
import pandas as pd
import re

In [2]:
GLOVE_PATH= "./glovo_6B"
FILEPATH = "./shoes_v2.csv"

In [26]:
words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=f'glove_6B/glovo.6B.50d.dat', mode='w')

with open(f'glove_6B/glove.6B.50d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
vectors = bcolz.carray(vectors[1:].reshape((400000, 50)), rootdir=f'/home/gr/UT/LIP/project/glove_6B/glovo.6B.50d.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'/home/gr/UT/LIP/project/glove_6B/glovo.6B.50d_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'/home/gr/UT/LIP/project/glove_6B/glovo.6B.50d_idx.pkl', 'wb'))

In [27]:
vectors = bcolz.open(f'glove_6B/glovo.6B.50d.dat')[:]
words = pickle.load(open(f'glove_6B/glovo.6B.50d_words.pkl', 'rb'))
word2idx = pickle.load(open(f'glove_6B/glovo.6B.50d_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [5]:
shoes = pd.read_csv('shoes_v2.csv')

In [6]:
shoes.head()

,Unnamed: 0,CID,Category,SubCategory,HeelHeight,Insole,Closure,Gender,Material,ToeStyle,Brand,Url,unique_Closure,unique_Gender,unique_Material
0,0,100627-72,Shoes,Oxfords,NaN,Leather,['Lace up'],['Men'],['Leather'],Capped Toe;Round Toe,Bostonian,Shoes/Oxfords/100627.72.jpg,Lace up,Men,Leather
1,1,100627-255,Shoes,Oxfords,NaN,Leather,['Lace up'],['Men'],['Leather'],Capped Toe;Round Toe,Bostonian,Shoes/Oxfords/100627.255.jpg,Lace up,Men,Leather
2,2,100657-72,Shoes,Oxfords,NaN,Leather;Padded;Removable,['Lace up'],['Men'],['Leather'],Capped Toe;Round Toe,Bostonian,Shoes/Oxfords/100657.72.jpg,Lace up,Men,Leather
3,3,100657-216,Shoes,Oxfords,NaN,Leather;Padded;Removable,['Lace up'],['Men'],['Leather'],Capped Toe;Round Toe,Bostonian,Shoes/Oxfords/100657.216.jpg,Lace up,Men,Leather
4,4,101026-3,Boots,Mid-Calf,1in - 1 3/4in,Leather;Padded,['Pull-on'],['Men'],"['Leather', 'Rubber']",Square Toe;Closed Toe,Durango,Boots/Mid-Calf/101026.3.jpg,Pull-on,Men,Leather


In [7]:
shoes = shoes.fillna("")

In [170]:
text = ""
for col in ['SubCategory', 'Brand', 'unique_Closure', 'unique_Gender', 'unique_Material']:
    tmp = " ".join(shoes[col].unique().tolist())
    text += tmp
text = text.split()
text = [t.lower() for t in text]

In [171]:
text

['oxfords',
 'mid-calf',
 'heel',
 'ankle',
 'boot',
 'loafers',
 'slipper',
 'flats',
 'boat',
 'shoes',
 'flat',
 'clogs',
 'and',
 'mules',
 'flats',
 'sneakers',
 'and',
 'athletic',
 'shoes',
 'slipper',
 'heels',
 'heels',
 'athletic',
 'prewalker',
 'knee',
 'high',
 'prewalker',
 'boots',
 'crib',
 'shoes',
 'firstwalker',
 'over',
 'the',
 'kneebostonian',
 'durango',
 'finn',
 'comfort',
 'frye',
 'hush',
 'puppies',
 'minnetonka',
 'nunn',
 'bush',
 'skechers',
 'sperry',
 'top-sider',
 'stacy',
 'adams',
 'bass',
 'wolky',
 'hush',
 'puppies',
 'kids',
 'klogs',
 'usa',
 'vans',
 'ecco',
 'florsheim',
 'sebago',
 'propet',
 'dr.',
 'martens',
 'birkenstock',
 "birki's",
 'l.b.',
 'evans',
 'softwalk',
 'daniel',
 'green',
 'easy',
 'spirit',
 'clarks',
 'reebok',
 'lifestyle',
 'rockport',
 'new',
 'balance',
 'deer',
 'stags',
 'ugg',
 'franco',
 'sarto',
 'caterpillar',
 'lugz',
 'wolverine',
 'converse',
 'old',
 'friend',
 'k-swiss',
 'dockers',
 'keds',
 'sesto',
 'meu

In [188]:
for t in text:
    if t.startswith("bos"):
        print(t)

boss
boss
boss


In [37]:
cut_df = shoes[['SubCategory', 'Brand', 'unique_Closure', 'unique_Gender', 'unique_Material']]

In [38]:
sent_list = cut_df.values.tolist()
sent_list = [' '.join(t) for t in sent_list]
sent_list = [re.sub(r'[^\w]', ' ', t) for t in sent_list]

In [39]:
sent_list = [t.split() for t in sent_list]
sent_list = [[s.lower() for s in t] for t in sent_list]
sent_list

[['oxfords', 'bostonian', 'lace', 'up', 'men', 'leather'],
 ['oxfords', 'bostonian', 'lace', 'up', 'men', 'leather'],
 ['oxfords', 'bostonian', 'lace', 'up', 'men', 'leather'],
 ['oxfords', 'bostonian', 'lace', 'up', 'men', 'leather'],
 ['mid', 'calf', 'durango', 'pull', 'on', 'men', 'leather'],
 ['mid', 'calf', 'durango', 'pull', 'on', 'men', 'rubber'],
 ['heel', 'finn', 'comfort', 'buckle', 'women', 'leather'],
 ['heel', 'finn', 'comfort', 'buckle', 'women', 'nappa'],
 ['heel', 'finn', 'comfort', 'buckle', 'women', 'leather'],
 ['heel', 'finn', 'comfort', 'buckle', 'women', 'nappa'],
 ['heel', 'finn', 'comfort', 'buckle', 'women', 'leather'],
 ['heel', 'finn', 'comfort', 'buckle', 'women', 'nappa'],
 ['heel', 'finn', 'comfort', 'buckle', 'women', 'leather'],
 ['heel', 'finn', 'comfort', 'buckle', 'women', 'nappa'],
 ['heel', 'finn', 'comfort', 'buckle', 'women', 'leather'],
 ['heel', 'finn', 'comfort', 'buckle', 'women', 'nappa'],
 ['heel', 'finn', 'comfort', 'buckle', 'women', 'leat

In [51]:
sent_one_list = cut_df.values.flatten().tolist()
sent_one_list = ' '.join(sent_one_list)
sent_one_list = re.sub(r'[^\w]', ' ', sent_one_list)

In [52]:
sent_one_list = sent_one_list.split(' ')
sent_one_list = [t.lower() for t in sent_one_list]
sent_one_list_unique = list(set(sent_one_list))
sent_list_add = ['pad', 'bos', 'eos', 'unk']
sent_one_list_unique = sent_list_add + sent_one_list_unique
len(sent_one_list_unique)

893

In [53]:
sent_one_list_unique

['pad',
 'bos',
 'eos',
 'unk',
 '',
 'supra',
 'drop',
 'mates',
 'pull',
 'habitat',
 'zipper',
 'kai',
 'camuto',
 'prewalker',
 'prana',
 'white',
 'rebecca',
 'murphy',
 'spade',
 'ragg',
 'garmont',
 'sport',
 'soft',
 'favorite',
 'vinyl',
 'grain',
 'dc',
 'loves',
 'gabor',
 'kickers',
 'of',
 'roper',
 'umi',
 'just',
 'miz',
 'horse',
 'strap',
 'gringo',
 'pliner',
 'lacrosse',
 'lucky',
 'velvet',
 'over',
 'reef',
 'g',
 'joie',
 'coloriffics',
 'ferragamo',
 'giuseppe',
 'generic',
 'cashmere',
 'heelys',
 'exotic',
 'mountrek',
 'back',
 'vibram',
 'malibu',
 'rooster',
 'hawaii',
 'terry',
 'garcia',
 's',
 'wayne',
 'tiger',
 'onitsuka',
 'florsheim',
 'el',
 'wrap',
 'fitflop',
 'flex',
 'juicy',
 'bates',
 'comfort',
 'dingo',
 'gentle',
 'riding',
 'modern',
 'reebok',
 'distressed',
 'nike',
 'gabriella',
 'softspots',
 'spirit',
 'canvas',
 'laundry',
 'edelman',
 'rocha',
 'gypsy',
 'salvatore',
 'shearling',
 'miss',
 'mcqueen',
 'jamie',
 'little',
 'timberlan

In [28]:
word2idx['pad']

10109

In [29]:
word2idx['eos']

26828

In [30]:
word2idx['bos']

24471

In [34]:
word2idx['unk']

201534

In [54]:
matrix_len = len(sent_one_list_unique)

weights_matrix = np.zeros((matrix_len, 50)) # <PAD> <EOS> <UNK>
words_found = 0
words_missing_count = 0
words_missing_list = []
word2idx = {word: i for i, word in enumerate(sent_one_list_unique)}

# Writing the weights_matrix
for i, word in enumerate(sent_one_list_unique):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        words_missing_count += 1
        words_missing_list.append(word)
        # This case is used for '<pad>' as a 500 element
        weights_matrix[i] = glove['unk']

In [55]:
words_missing_count

123

In [56]:
words_missing_list

['',
 'prewalker',
 'garmont',
 'coloriffics',
 'mountrek',
 'fitflop',
 'softspots',
 'pikolinos',
 'havaianas',
 'dansko',
 'propet',
 'earthies',
 'cienta',
 'fivefingers',
 'footmates',
 'robeez',
 'ziera',
 'ahnu',
 'lumiani',
 'olukai',
 'nubuck',
 'naot',
 'fitzwell',
 'pablosky',
 'fluevog',
 'hommerson',
 'softwalk',
 'fidji',
 'klogs',
 'sadock',
 'daniblack',
 'ariat',
 'zamberlan',
 'aravon',
 'sporto',
 'pediped',
 'orthaheel',
 'arcopedico',
 'ektelon',
 'bandolino',
 'cushe',
 'elephantito',
 'bedstu',
 'matiko',
 'bcbgeneration',
 'giesswein',
 'superfit',
 'naturalista',
 'livie',
 'blundstone',
 'zigi',
 'golite',
 'garvalin',
 'bcbgmaxazria',
 'aetrex',
 'venettini',
 'kamalikulture',
 'montrail',
 'biviel',
 'privo',
 'rosegold',
 'magnanni',
 'mooz',
 'freewaters',
 'lifestride',
 'emerica',
 'inov',
 'wolky',
 'vaneli',
 'kalso',
 'sanita',
 'foamtreads',
 'pampili',
 'kamik',
 'pajar',
 'worishofer',
 'chooka',
 'beeko',
 'dknyc',
 'smartdogs',
 'flojos',
 'luich

In [16]:
# Shape of the weights matrix
NUM_WORDS, VOC_SIZE = weights_matrix.shape[0], weights_matrix.shape[1]

# ADD 'bos' and 'eos' to sent_list and 'pad'

emb_sent=[]
for sent in sent_list:
    emb_sent.append([word2idx[i] for i in sent])
    

print(NUM_WORDS, VOC_SIZE)
print(emb_sent[0])

889 50
[775, 221, 175, 758, 756, 131]


In [17]:
weights_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.67446 , -0.34052 ,  0.12976 , ...,  0.21709 ,  0.95675 ,
         0.13755 ],
       [-0.24837 , -0.463   ,  0.82342 , ...,  0.34312 ,  0.52278 ,
        -0.046782],
       ...,
       [ 0.20649 ,  0.52208 ,  0.39041 , ..., -0.81835 ,  0.065401,
         0.033825],
       [-0.13634 ,  0.43304 , -0.10769 , ...,  0.48265 ,  0.5756  ,
         0.88982 ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]])